In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns

# paths - assuming this notebook is in /analysis
ROOT = Path("..").resolve()
DATA_RAW   = ROOT / "data" / "raw" / "housing.csv"
DATA_TRAIN = ROOT / "data" / "train"
DATA_TEST  = ROOT / "data" / "test"
IMAGES     = ROOT / "images"

DATA_TRAIN.mkdir(parents=True, exist_ok=True)
DATA_TEST.mkdir(parents=True, exist_ok=True)
IMAGES.mkdir(parents=True, exist_ok=True)
